In [1]:
import pandas as pd
import numpy as np

# Линейная регрессия

Линейная регрессия позволяет установить зависимость между факторами и зависимой переменной. Значение зависимой переменной будет выражаться следующим образом: $y = f(x_1, x_2, \dots, x_n) + \epsilon$, где $\epsilon$ - остаток, не зависящий от факторов.

Простая линейная регрессия выглядит следующим образом: $y_i = \beta_0 + \beta_1 x_i + \epsilon_i$. Мы также предполагаем, что матожидание остатков равно 0, а дисперсия постоянна и нет корреляции между остатками для различных элементов выборки (т.е. остатки не описываются неким неучтенным нами фактором). 

Для нахождения параметров $\beta_0$ и $\beta_1$ используется метод наимньших квадратов, в котором необходимо минимизировать следующее выражение: $\sum(y_i - \beta_0 - \beta_1 x_i)^2 \rightarrow min$. Для его минимизации найдем частные производные по всем переменным и составим систему уравнений, из которой получим следующие формулы для вычисления параметров:
$$\hat{\beta}_1 = \frac{\sum(y_i x_i - \bar{y}\bar{x})}{\sum(x^2_i - \bar{x}^2)}$$
$$\hat{\beta}_0 = \bar{y} - \hat{\beta}_1x$$

Также вычисляется RSS (сумма квадратов остатков) для оценки общей ошибки и того, насколько факторами описывается поведение зависимой переменной.

#### Пример

In [2]:
df = pd.read_csv('car_regr.txt', sep='\t')
df.head()

,price,year,mileage,auto
0,250,2010,67.0,MT
1,365,2013,59.0,MT
2,365,2013,59.0,MT
3,250,2009,95.0,MT
4,310,2011,76.5,MT


In [3]:
# строим зависимость цены от года
X = df.year.values
Y = df.price.values

mX = X.mean()
mY = Y.mean()

In [4]:
# вычисляем параметры
B1 = sum(X * Y - mX*mY) / sum(X**2 - mX**2)
print(f"B1 = {B1}")

B0 = mY - B1 * mX
print(f"B0 = {B0}")

B1 = 26.043399374849457
B0 = -52090.26932084801


In [5]:
# строим саму модель
y_hat = lambda x: B0 + B1 * x
y_hat = np.vectorize(y_hat)

In [6]:
# вычисляем остатки
RSS = sum((Y - y_hat(X))**2)
print(f"RSS = {RSS}")

# дисперсия
S2 = RSS / (len(df) - 2)
print(f"S2 = {S2}")

# R^2 коэф
R2 = RSS / sum((Y - mY)**2)
R2 = 1 - R2
print(R2)

RSS = 132015.3192539345
S2 = 3143.2218869984404
0.6167004931126856


## Линейная регрессия от нескольких факторов

Точное аналитическое решение выглядит следующим образом:
$$\beta = (A^TA)^{-1}A^TY$$, где A - матрица факторов с добавленным первым столбцом из единиц, а Y - матрица Nx1 ответов.

In [7]:
# преобразуем категориальный признак в числовой
df['auto'] = (df["auto"] == 'MT').astype(int)

In [8]:
# берем матрицу ответов
Y = df.price.values
Y[:5]

array([250, 365, 365, 250, 310], dtype=int64)

In [9]:
# и матрицу факторов с добавленным столбцом единиц
X = df[['year', 'mileage', 'auto']].values
tX = np.ones((len(df), len(df.columns)))
tX[:, 1:] = X
tX[:5]

array([[1.000e+00, 2.010e+03, 6.700e+01, 1.000e+00],
       [1.000e+00, 2.013e+03, 5.900e+01, 1.000e+00],
       [1.000e+00, 2.013e+03, 5.900e+01, 1.000e+00],
       [1.000e+00, 2.009e+03, 9.500e+01, 1.000e+00],
       [1.000e+00, 2.011e+03, 7.650e+01, 1.000e+00]])

In [10]:
# находим аналитическое решение
beta = np.linalg.inv((tX.transpose() @ tX)) @ tX.transpose() @ Y
beta

array([-4.08225374e+04,  2.04883615e+01, -1.31397141e-01, -1.03582723e+02])

In [11]:
# строим модель
def y_hat(params) -> float:
    return beta[0] + beta[1] * params[0] + beta[2] * params[1] + beta[3] * params[2]

In [12]:
# вычисляем остатки
Y_hat = np.array([y_hat(x) for x in X])
RSS = sum((Y - Y_hat)**2)
print(f"RSS = {RSS}")

# дисперсия
S2 = RSS / (len(df) - X.shape[1] - 1)
print(f"S2 = {S2}")

# R^2 коэф
R2 = RSS / sum((Y - mY)**2)
R2 = 1 - R2
print(R2)

RSS = 87587.49036165551
S2 = 2189.6872590413877
0.745694347785934
